In [2]:
import os
from tkinter import filedialog

In [3]:
path = filedialog.askopenfilename()
exefile = '/opt/homebrew/anaconda3/envs/hwp-parser-test/bin/hwp5proc'

In [4]:
mXml = rf'{path[:-4]}.xml'
mHwp = path
cmd = rf'{exefile} xml "{mHwp}" > "{mXml}"'

os.system(cmd)

unmatched field end


0

In [9]:
import lxml.etree as ET
import re

In [18]:
tree = ET.parse(mXml)
root = tree.getroot()

search_text = "일자별 발파 및 계측 현황"

find_tag = []
for elem in root.iter():
    if elem.text and search_text in elem.text:
        print(elem.text)
        for found_tag in elem.iterancestors():
            if found_tag.tag == "ColumnSet":
                find_tag.append(found_tag)



6.5. 일자별 발파 및 계측 현황
일자별 발파 및 계측 현황


In [20]:

table_data = []
current_table_data = []
for cell in find_tag[1].findall(".//TableCell"):
    cell_data = {
        "row": cell.get("row"),
        "col": cell.get("col"),
        "text": "".join(elem.text for elem in cell.findall(".//Text") if elem.text)
    }
    print(cell_data)

    if len(current_table_data) != 0 and cell_data['row'] == '0' and cell_data['col'] == '0':
        table_data.append(current_table_data)
        current_table_data = [cell_data]
    else:
        current_table_data.append(cell_data)

print(table_data)

{'row': '0', 'col': '0', 'text': ''}
{'row': '0', 'col': '1', 'text': ''}
{'row': '0', 'col': '0', 'text': '일시'}
{'row': '0', 'col': '1', 'text': '발파횟수'}
{'row': '0', 'col': '2', 'text': '시간'}
{'row': '0', 'col': '3', 'text': '장약량(kg)'}
{'row': '0', 'col': '5', 'text': '발파위치'}
{'row': '0', 'col': '6', 'text': '발파진동 및 소음 측정치(max값)'}
{'row': '1', 'col': '3', 'text': '지발당 장약량'}
{'row': '1', 'col': '4', 'text': '총장약량'}
{'row': '1', 'col': '5', 'text': 'STA'}
{'row': '1', 'col': '6', 'text': '발파진동(cm/s)'}
{'row': '1', 'col': '7', 'text': '진동레벨dB(V)'}
{'row': '1', 'col': '8', 'text': '소음레벨dB(A)'}
{'row': '1', 'col': '9', 'text': '측정위치'}
{'row': '2', 'col': '0', 'text': '3월1일'}
{'row': '2', 'col': '1', 'text': '1회'}
{'row': '2', 'col': '2', 'text': '7:01'}
{'row': '2', 'col': '3', 'text': '1.6-2.0'}
{'row': '2', 'col': '4', 'text': '300.0'}
{'row': '2', 'col': '5', 'text': '7k+771.25~774.25(종점)'}
{'row': '2', 'col': '6', 'text': 'N/T'}
{'row': '2', 'col': '7', 'text': 'N/T'}
{'row': '2', 'col

In [21]:
columns = ['일자', '계측위치', '진동레벨', '소음레벨']

filtered_data = [
    sublist for sublist in table_data
    if any(
        entry['row'] in ['0', '1'] and any(keyword in entry['text'] for keyword in columns)
        for entry in sublist
    )
]

In [155]:
filtered_data

[[{'row': '0', 'col': '0', 'text': '일 자'},
  {'row': '0', 'col': '1', 'text': '계측위치'},
  {'row': '0', 'col': '2', 'text': '발파횟수'},
  {'row': '0', 'col': '3', 'text': '시간'},
  {'row': '0', 'col': '4', 'text': '관리기준'},
  {'row': '0', 'col': '7', 'text': '측정결과'},
  {'row': '1', 'col': '4', 'text': '진동속도(cm/s)'},
  {'row': '1', 'col': '5', 'text': '진동레벨(dB(V))'},
  {'row': '1', 'col': '6', 'text': '소음레벨(dB(A))'},
  {'row': '1', 'col': '7', 'text': '진동속도(cm/s)'},
  {'row': '1', 'col': '8', 'text': '진동레벨(dB(V))'},
  {'row': '1', 'col': '9', 'text': '소음레벨(dB(A))'},
  {'row': '2', 'col': '0', 'text': '3월1일'},
  {'row': '2', 'col': '1', 'text': '다인모듈'},
  {'row': '2', 'col': '2', 'text': '1회'},
  {'row': '2', 'col': '3', 'text': '7:00'},
  {'row': '2', 'col': '4', 'text': '0.2'},
  {'row': '2', 'col': '5', 'text': '75'},
  {'row': '2', 'col': '6', 'text': '75'},
  {'row': '2', 'col': '7', 'text': '0.053'},
  {'row': '2', 'col': '8', 'text': '56.46'},
  {'row': '2', 'col': '9', 'text': '53.23'},

In [156]:
os.remove(mXml)